In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=d04634c2-39f5-455e-9c89-0916542bfa46
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:04<00:00, 253MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from models import incepres_model
from models import CompModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 8104.53it/s]


In [5]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)

100%|██████████| 10344/10344 [00:00<00:00, 785677.20it/s]
10344it [00:08, 1208.65it/s]


In [6]:
x_train, y_train, x_test, y_test = including_data(labels, ds)

10344it [00:04, 2147.61it/s]


In [7]:
# transformation and resizing
x_train = RP_final_data(x_train)

100%|██████████| 5000/5000 [24:29<00:00,  3.40it/s]


In [8]:
model = incepres_model()

In [9]:
model, lr_sch = CompModel(model)

history = model.fit(x_train, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100
157/157 [==============================] - 62s 282ms/step - loss: 0.5641 - acc: 0.5026 - lr: 0.0010
Epoch 2/100
157/157 [==============================] - 43s 277ms/step - loss: 0.3611 - acc: 0.6200 - lr: 9.7724e-04
Epoch 3/100
157/157 [==============================] - 43s 277ms/step - loss: 0.2774 - acc: 0.6304 - lr: 9.5499e-04
Epoch 4/100
157/157 [==============================] - 43s 277ms/step - loss: 0.2566 - acc: 0.6462 - lr: 9.3325e-04
Epoch 5/100
157/157 [==============================] - 43s 276ms/step - loss: 0.2500 - acc: 0.6480 - lr: 9.1201e-04
Epoch 6/100
157/157 [==============================] - 43s 277ms/step - loss: 0.2463 - acc: 0.6544 - lr: 8.9125e-04
Epoch 7/100
157/157 [==============================] - 43s 276ms/step - loss: 0.2403 - acc: 0.6556 - lr: 8.7096e-04
Epoch 8/100
157/157 [==============================] - 43s 276ms/step - loss: 0.2369 - acc: 0.6582 - lr: 8.5114e-04
Epoch 9/100
157/157 [==============================] - 43s 275ms/step - loss

In [11]:
model.save('/kaggle/working/incepres_model.h5')